In [1]:
# In Terminal, "pip install ibis-framework[duckdb] pyjanitor"
import pandas as pd
import ibis
from ibis import selectors as s
from ibis import _
ibis.options.interactive = True

In [2]:
from janitor import clean_names

In [3]:
# Path
from pathlib import Path
path = Path("~/datasets/home-dataset/jupyterlab/ZX05")
db_path = path / "db"
meta_path = path / "meta"
output_path = path / "output"

In [4]:
# Read DB file
df = pd.read_csv(db_path / "CF_2024.csv", dtype={"costctr":str})

In [5]:
cf = ibis.memtable(df)
cf.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual   ┃ plan     ┃ target   ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ int64    │ int64    │ int64    │
├───────┼────────┼─────────┼─────────────┼──────────┼──────────┼──────────┤
│  2024 │      1 │ 4110    │ K082        │    35620 │        0 │        0 │
│  2024 │      1 │ 4110    │ K30001      │ 22040000 │ 21880000 │ 21880000 │
│  2024 │      1 │ 4110    │ K30002      │   170900 │   748516 │   748516 │
└───────┴────────┴─────────┴─────────────┴──────────┴──────────┴──────────┘

In [6]:
# Process numeric columns
cf = (cf
    .mutate(s.across(["actual", "plan", "target"], _ / -1000))
    .mutate(delta_to_plan = (_.actual - _.plan).round(3))
)    
cf.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual    ┃ plan       ┃ target     ┃ delta_to_plan ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64   │ float64    │ float64    │ float64       │
├───────┼────────┼─────────┼─────────────┼───────────┼────────────┼────────────┼───────────────┤
│  2024 │      1 │ 4110    │ K082        │    -35.62 │     -0.000 │     -0.000 │       -35.620 │
│  2024 │      1 │ 4110    │ K30001      │ -22040.00 │ -21880.000 │ -21880.000 │      -160.000 │
│  2024 │      1 │ 4110    │ K30002      │   -170.90 │   -748.516 │   -748.516 │       577.616 │
└───────┴────────┴─────────┴─────────────┴───────────┴────────────┴────────────┴───────────────┘

### Read master data

In [7]:
# Read master data
df_acc = pd.read_csv(meta_path / "0000_TABLE_MASTER_Acc level.csv", dtype=str).clean_names()
df_cc_general = pd.read_csv(meta_path / "0000_TABLE_MASTER_Cost center_general.csv", dtype=str).clean_names()
df_cc_hierarchy = pd.read_csv(meta_path / "0000_TABLE_MASTER_Cost center_hierarchy.csv", dtype=str).clean_names()

In [8]:
acc = ibis.memtable(df_acc)
acc.head(3)

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ account_no_ ┃ account_name         ┃ account_description         ┃ acc_lv1                                ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string      │ string               │ string                      │ string                                 │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │
├─────────────┼──────────────────────┼─────────────────────────────┼────────────────────────────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┤
│ K275        │ Vacation payments    │ K275 Vacation payments      │ 158 Social benefit rate wages variable │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │
│ K28401      │ Company pension - w  │ K28401 Company pension - w  │ 158 Social benefit rate wages variable │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │
│ K28811      │ Soc.sec. pens.fund w │ K28811 Soc.sec. pens.fund w │ 158 Social benefit rate wages variable │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │
└─────────────┴──────────────────────┴─────────────────────────────┴────────────────────────────────────────┴─────────────────────────┴───────────────────────┴───────────────────────────┴──────────────┴─────────────────────────────┴─────────────────────────────┴───────────────────────┴───────────────┘

In [9]:
cc_general = ibis.memtable(df_cc_general)
cc_general.head(3)

┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ cctr   ┃ validity ┃ pctr      ┃ fix_var ┃ cctr_description         ┃ responsible   ┃ manning_type ┃ line_type ┃ department ┃ function ┃ ordera ┃ orderb ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ string │ string   │ string    │ string  │ string                   │ string        │ string       │ string    │ string     │ string   │ string │ string │
├────────┼──────────┼───────────┼─────────┼──────────────────────────┼───────────────┼──────────────┼───────────┼────────────┼──────────┼────────┼────────┤
│ 36098  │ Valid    │ 50803-010 │ Var     │ E CT DTC SMD Technician  │ Yang, JunHyuk │ HEF          │ SMD       │ Production │ SMD      │ J      │ JA     │
│ 38899  │ Valid    │ 50803-026 │ Fix     │ P HT HYD Line share cost │ Kim, Hagjae   │ NULL         │ NULL      │ Production │ FF1      │ J      │ JB     │
│ 34855  │ Valid    │ 50803-034 │ Fix     │ P SC MES ICH Dummy       │ Seo, Jiyun    │ NULL         │ NULL      │ Production │ FF1      │ J      │ JB     │
└────────┴──────────┴───────────┴─────────┴──────────────────────────┴───────────────┴──────────────┴───────────┴────────────┴──────────┴────────┴────────┘

In [10]:
cc_hierarchy = ibis.memtable(df_cc_hierarchy)
cc_hierarchy.head(3)

┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ cctr   ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3      ┃ lv4        ┃ lv5    ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ string │ string │ string │ string  │ string     │ string   │ string     │ string │
├────────┼────────┼────────┼─────────┼────────────┼──────────┼────────────┼────────┤
│ 4311   │ 45292  │ 45657  │ IC-OICP │ ICLM-O7811 │ ICLM-MGK │ ICLM-MGK-A │ NULL   │
│ 4321   │ 45292  │ 45657  │ IC-OICP │ ICLM-O7811 │ ICLM-MGK │ ICLM-MGK-A │ NULL   │
│ 4830   │ 45292  │ 45657  │ IC-OICP │ ICLM-O7811 │ ICLM-MGK │ ICLM-MGK-A │ NULL   │
└────────┴────────┴────────┴─────────┴────────────┴──────────┴────────────┴────────┘

In [11]:
cc = cc_general.join(cc_hierarchy, "cctr", how="left")
cc.head(3)

┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ cctr   ┃ validity ┃ pctr      ┃ fix_var ┃ cctr_description         ┃ responsible   ┃ manning_type ┃ line_type ┃ department ┃ function ┃ ordera ┃ orderb ┃ cctr_right ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3      ┃ lv4        ┃ lv5    ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ string │ string   │ string    │ string  │ string                   │ string        │ string       │ string    │ string     │ string   │ string │ string │ string     │ string │ string │ string  │ string     │ string   │ string     │ string │
├────────┼──────────┼───────────┼─────────┼──────────────────────────┼───────────────┼──────────────┼───────────┼────────────┼──────────┼────────┼────────┼────────────┼────────┼────────┼─────────┼────────────┼──────────┼────────────┼────────┤
│ 36098  │ Valid    │ 50803-010 │ Var     │ E CT DTC SMD Technician  │ Yang, JunHyuk │ HEF          │ SMD       │ Production │ SMD      │ J      │ JA     │ 36098      │ 45292  │ 45657  │ IC-OICP │ ICLD-O7821 │ ICLD-FGK │ ICLD-FGK-S │ NULL   │
│ 38899  │ Valid    │ 50803-026 │ Fix     │ P HT HYD Line share cost │ Kim, Hagjae   │ NULL         │ NULL      │ Production │ FF1      │ J      │ JB     │ 38899      │ 45292  │ 45657  │ IC-OICP │ ICEY-O7411 │ ICEY-FGK │ ICEY-FGK-S │ NULL   │
│ 34855  │ Valid    │ 50803-034 │ Fix     │ P SC MES ICH Dummy       │ Seo, Jiyun    │ NULL         │ NULL      │ Production │ FF1      │ J      │ JB     │ 34855      │ 45292  │ 45657  │ IC-OICP │ ICMK-O7521 │ ICMK-FGK │ ICMK-FGK-S │ NULL   │
└────────┴──────────┴───────────┴─────────┴──────────────────────────┴───────────────┴──────────────┴───────────┴────────────┴──────────┴────────┴────────┴────────────┴────────┴────────┴─────────┴────────────┴──────────┴────────────┴────────┘

### Data wrangling

In [12]:
# Join tables to get master data
cf = (cf
    .join(acc, cf.gl_accounts == acc.account_no_, how="left")
    .join(cc, cf.costctr == cc.cctr, how="left")
    .drop("cctr", "account_no_", "cctr_right")
)
cf.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual   ┃ plan       ┃ target     ┃ delta_to_plan ┃ account_name         ┃ account_description       ┃ acc_lv1            ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃ validity ┃ pctr      ┃ fix_var ┃ cctr_description    ┃ responsible ┃ manning_type ┃ line_type ┃ department ┃ function   ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1       ┃ lv2        ┃ lv3      ┃ lv4    ┃ lv5    ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64  │ float64    │ float64    │ float64       │ string               │ string                    │ string             │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │ string   │ string    │ string  │ string              │ string      │ string       │ string    │ string     │ string     │ string │ string │ string │ string │ string    │ string     │ string   │ string │ string │
├───────┼────────┼─────────┼─────────────┼──────────┼────────────┼────────────┼───────────────┼──────────────────────┼───────────────────────────┼────────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┼──────────┼───────────┼─────────┼─────────────────────┼─────────────┼──────────────┼───────────┼────────────┼────────────┼────────┼────────┼────────┼────────┼───────────┼────────────┼──────────┼────────┼────────┤
│  2024 │      1 │ 4110    │ K30001      │ -22040.0 │ -21880.000 │ -21880.000 │      -160.000 │ Salary               │ K30001 Salary             │ 275 Total salaries │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ 50899-999 │ Fix     │ CF Plant Purchasing │ Seo, Jeewon │ NEF          │ NULL      │ Purchasing │ Purchasing │ J      │ JG     │ 45292  │ 45657  │ ICZZ-ZICP │ ICZZ-PLANT │ ICZZ-MGK │ NULL   │ NULL   │
│  2024 │      1 │ 4110    │ K30002      │   -170.9 │   -748.516 │   -748.516 │       577.616 │ Overtime salaries    │ K30002 Overtime salaries  │ 275 Total salaries │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ 50899-999 │ Fix     │ CF Plant Purchasing │ Seo, Jeewon │ NEF          │ NULL      │ Purchasing │ Purchasing │ J      │ JG     │ 45292  │ 45657  │ ICZZ-ZICP │ ICZZ-PLANT │ ICZZ-MGK │ NULL   

In [13]:
# Add account information
cf = cf.mutate(
    ce_text = ibis.case()
        # E01-299
        .when(cf.acc_lv2 == "299 Total Labor Costs", "10_Compensation")
        # E01-465
        .when((cf.acc_lv1 == "345 Depreciation long life") | (cf.acc_lv1 == "370 Rental/Leasing"), "11_Depreciation & Leasing")
        .when(cf.acc_lv1 == "375 Utilities", "12_Energy")
        .when(cf.acc_lv1 == "435 Fees and purchased services", "13_Fees and Purchased Services")
        .when(cf.acc_lv1 == "320 Purchased maintenance", "15_Maintenance")
        .when((cf.acc_lv1 == "430 Entertainment/Travel expense") | (cf.acc_lv1 == "440 Recruitment/Training/Development"), "16_Travel Training")
        .when(cf.acc_lv2 == "465 Cost of materials", "17_Other cost")
        # E01-535
        .when(cf.gl_accounts == "K6620", "18_Services In / Out")
        .when(cf.gl_accounts == "K6626", "19_Transfer out")
        .when((cf.gl_accounts == "K6623") | (cf.gl_accounts == "K6624") | (cf.gl_accounts == "K6625"), "20_IT Allocation")
        # E01-520
        .when(cf.acc_lv2 == "520 Assessments In", "CF cost assessment out")
        .when(True, "NA").end()
)
cf.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual   ┃ plan       ┃ target     ┃ delta_to_plan ┃ account_name         ┃ account_description       ┃ acc_lv1            ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃ validity ┃ pctr      ┃ fix_var ┃ cctr_description    ┃ responsible ┃ manning_type ┃ line_type ┃ department ┃ function   ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1       ┃ lv2        ┃ lv3      ┃ lv4    ┃ lv5    ┃ ce_text         ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64  │ float64    │ float64    │ float64       │ string               │ string                    │ string             │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │ string   │ string    │ string  │ string              │ string      │ string       │ string    │ string     │ string     │ string │ string │ string │ string │ string    │ string     │ string   │ string │ string │ string          │
├───────┼────────┼─────────┼─────────────┼──────────┼────────────┼────────────┼───────────────┼──────────────────────┼───────────────────────────┼────────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┼──────────┼───────────┼─────────┼─────────────────────┼─────────────┼──────────────┼───────────┼────────────┼────────────┼────────┼────────┼────────┼────────┼───────────┼────────────┼──────────┼────────┼────────┼─────────────────┤
│  2024 │      1 │ 4110    │ K30001      │ -22040.0 │ -21880.000 │ -21880.000 │      -160.000 │ Salary               │ K30001 Salary             │ 275 Total salaries │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ 50899-999 │ Fix     │ CF Plant Purchasing │ Seo, Jeewon │ NEF          │ NULL      │ Purchasing │ Purchasing │ J      │ JG     │ 45292  │ 45657  │ ICZZ-ZICP │ ICZZ-PLANT │ ICZZ-MGK │ NULL   │ NULL   │ 10_Compensation │
│  2024 │      1 │ 4110    │ K30002      │   -170.9 │   -748.516 │   -748.516 │       577.616 │ Overtime salaries    │ K30002 Overtime salaries  │ 275 Total salaries │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ 50899-999 │ Fix     │ CF Plant Purchasing │ Seo, Jeewon │ NEF          │ NULL     

In [14]:
# Remove unnecessary columns
cf = cf.drop(
            "validity", "responsible", "account_description", "acc_lv1_by_consolidated",
            "acc_lv3", "acc_lv4", "acc_lv5", "acc_lv6",
            )
cf.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual   ┃ plan       ┃ target     ┃ delta_to_plan ┃ account_name         ┃ acc_lv1            ┃ acc_lv2               ┃ general_lv1           ┃ pmme_category ┃ pctr      ┃ fix_var ┃ cctr_description    ┃ manning_type ┃ line_type ┃ department ┃ function   ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1       ┃ lv2        ┃ lv3      ┃ lv4    ┃ lv5    ┃ ce_text         ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64  │ float64    │ float64    │ float64       │ string               │ string             │ string                │ string                │ string        │ string    │ string  │ string              │ string       │ string    │ string     │ string     │ string │ string │ string │ string │ string    │ string     │ string   │ string │ string │ string          │
├───────┼────────┼─────────┼─────────────┼──────────┼────────────┼────────────┼───────────────┼──────────────────────┼────────────────────┼───────────────────────┼───────────────────────┼───────────────┼───────────┼─────────┼─────────────────────┼──────────────┼───────────┼────────────┼────────────┼────────┼────────┼────────┼────────┼───────────┼────────────┼──────────┼────────┼────────┼─────────────────┤
│  2024 │      1 │ 4110    │ K30001      │ -22040.0 │ -21880.000 │ -21880.000 │      -160.000 │ Salary               │ 275 Total salaries │ 299 Total Labor Costs │ 299 Total Labor Costs │ Compensation  │ 50899-999 │ Fix     │ CF Plant Purchasing │ NEF          │ NULL      │ Purchasing │ Purchasing │ J      │ JG     │ 45292  │ 45657  │ ICZZ-ZICP │ ICZZ-PLANT │ ICZZ-MGK │ NULL   │ NULL   │ 10_Compensation │
│  2024 │      1 │ 4110    │ K30002      │   -170.9 │   -748.516 │   -748.516 │       577.616 │ Overtime salaries    │ 275 Total salaries │ 299 Total Labor Costs │ 299 Total Labor Costs │ Compensation  │ 50899-999 │ Fix     │ CF Plant Purchasing │ NEF          │ NULL      │ Purchasing │ Purchasing │ J      │ JG     │ 45292  │ 45657  │ ICZZ-ZICP │ ICZZ-PLANT │ ICZZ-MGK │ NULL   │ NULL   │ 10_Compensation │
│  2024 │      1 │ 4110    │ K359        │   -270.0 │   -270.000 │   -270.000 │         0.000 │ Monetary Benef.-Cars │ 275 Total salaries │ 299 Total Labor Costs │ 299 Total Labor Costs │ Compensation  │ 50899-999 │ Fix     │ CF Plant Purchasing │ NEF          │ NULL      │ Purchasing │ Purchasing │ J      │ JG     │ 45292  │ 45657  │ ICZZ-ZICP │ ICZZ-PLANT │ ICZZ-MGK │ NULL   │ NULL   │ 10_Compensation │
└───────┴────────┴─────────┴─────────────┴──────────┴────────────┴────────────┴───────────────┴──────────────────────┴────────────────────┴───────────────────────┴───────────────────────┴───────────────┴───────────┴─────────┴─────────────────────┴──────────────┴───────────┴────────────┴────────────┴────────┴────────┴────────┴────────┴───────────┴────────────┴──────────┴────────┴────────┴─────────────────┘

### Output data

In [15]:
# Output data
cf.to_pandas().to_csv(output_path / "CF costs.csv", index=False)